<a href="https://colab.research.google.com/github/ahsanrazi/LangChain/blob/main/11_Multivector_Retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY').strip()

In [2]:
!pip install -qU langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.4 MB/s eta 0:00:00


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-exp", api_key = gemini_api_key)

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key = gemini_api_key)

In [5]:
!pip install -qU langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 3.5 MB/s eta 0:00:00


In [53]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

index_name = "langchain"
namespace = "Multivector_Retriever_Summary"

pc = Pinecone(api_key= userdata.get('PINECONE_API'))
index = pc.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings, index=index, namespace=namespace)

In [9]:
!pip install -qU langchain langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [41]:
from langchain.storage import InMemoryByteStore
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loaders = [
    TextLoader("/content/Bot Name Jasmine.txt")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(docs)

In [44]:
docs

[Document(metadata={'source': '/content/Bot Name Jasmine.txt'}, page_content='We Build Trades is a UK-based digital marketing and software agency founded in 2017 by Daniel Brown. \nIt specializes in comprehensive marketing solutions for trade businesses, helping clients manage sales, \nmarketing, and operations effectively through innovative tools and strategies.\nServices\nWe Build Trades offers tailored solutions for ambitious trade business owners in the UK, Ireland, the US, \nand Canada. Their key services include:\n1. Website Design\no Creation of custom, conversion-focused websites to help businesses stand out.\n2. Copywriting\no Professionally crafted copy showcasing industry expertise.\n3. Automated Systems\no Tools like Trades Gang to streamline lead follow-ups and operations.\n4. Data & Tracking\no Proprietary analytics systems to measure and report key business metrics.\n5. Paid Advertising\no Customized marketing campaigns on platforms such as Google and Facebook.\n6. SEO S

In [71]:
len(docs)

8

In [45]:
import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever

# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vector_store,
    byte_store=store,
    id_key=id_key,
)

doc_ids = [str(uuid.uuid4()) for _ in docs]

In [72]:
doc_ids

['0b537407-338a-400c-9299-269dfeda113d',
 'b512ab84-93b2-4ed0-8b72-9b1d9b252c60',
 '27e4d90a-f2aa-487c-bd6f-0f3ea264496b',
 'be8df8be-440d-4dc2-a2cf-ef2fb6b85128',
 'a751febb-7a85-4ca3-ab3d-d6aaaa293817',
 '09e20cd7-af95-4c56-b843-2eea046b4ad0',
 '674f3a5e-1bc5-47ad-913a-db9253d86f0d',
 '56b6c743-527f-4ce8-ac37-8e56edf3a539']

In [46]:
# The splitter to use to create smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=750)

sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

In [47]:
len(sub_docs)

15

In [48]:
sub_docs

[Document(metadata={'source': '/content/Bot Name Jasmine.txt', 'doc_id': 'cd3178a9-2442-402b-af2b-7d512bc96a01'}, page_content='We Build Trades is a UK-based digital marketing and software agency founded in 2017 by Daniel Brown. \nIt specializes in comprehensive marketing solutions for trade businesses, helping clients manage sales, \nmarketing, and operations effectively through innovative tools and strategies.\nServices\nWe Build Trades offers tailored solutions for ambitious trade business owners in the UK, Ireland, the US, \nand Canada. Their key services include:\n1. Website Design\no Creation of custom, conversion-focused websites to help businesses stand out.\n2. Copywriting\no Professionally crafted copy showcasing industry expertise.\n3. Automated Systems\no Tools like Trades Gang to streamline lead follow-ups and operations.\n4. Data & Tracking'),
 Document(metadata={'source': '/content/Bot Name Jasmine.txt', 'doc_id': 'cd3178a9-2442-402b-af2b-7d512bc96a01'}, page_content='2.

In [49]:
retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [51]:
retriever.vectorstore.similarity_search("Founder of We Build Trade")[0]

Document(id='14814ee9-418c-4fbb-8678-d7e35f2e2651', metadata={'doc_id': '4c0bf6b8-46e4-4065-a461-c922543e35a4', 'source': '/content/Bot Name Jasmine.txt'}, page_content='1. Client Obsession\no Always prioritizing what is best for clients.\n2. Full Transparency\no Maintaining honesty and openness in all dealings.\n3. Long-Term Thinking\no Emphasizing sustainable growth over short-term gains.\n4. Relentless Ambition\no Continuously striving for excellence and improvement.\nFounder: Daniel Brown\n• Role: Founder and CEO of We Build Trades.\n• Career Start: Former senior marketing consultant.')

In [52]:
import uuid

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | model
    | StrOutputParser()
)

In [54]:
summaries = chain.batch(docs, {"max_concurrency": 2})

In [55]:
summaries

['We Build Trades, founded in 2017, is a UK-based digital marketing and software agency specializing in providing comprehensive solutions for trade businesses in the UK, Ireland, the US, and Canada. They offer services like website design, copywriting, automated systems (including Trades Gang), data & tracking analytics, paid advertising, and SEO to help businesses manage sales, marketing, and operations effectively. Their focus is on creating custom, conversion-focused strategies to help trade businesses stand out and improve their overall performance.',
 'We Build Trades, founded by former senior marketing consultant Daniel Brown, offers services aimed at helping trades businesses grow. These services include paid advertising on platforms like Google and Facebook, SEO services to improve organic search rankings, professional graphics and video creation, job management through CRM integration, and coaching & support resources. The company operates with core values of client obsession,

In [73]:
len(summaries)

8

In [56]:
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vector_store,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [74]:
doc_ids

['0b537407-338a-400c-9299-269dfeda113d',
 'b512ab84-93b2-4ed0-8b72-9b1d9b252c60',
 '27e4d90a-f2aa-487c-bd6f-0f3ea264496b',
 'be8df8be-440d-4dc2-a2cf-ef2fb6b85128',
 'a751febb-7a85-4ca3-ab3d-d6aaaa293817',
 '09e20cd7-af95-4c56-b843-2eea046b4ad0',
 '674f3a5e-1bc5-47ad-913a-db9253d86f0d',
 '56b6c743-527f-4ce8-ac37-8e56edf3a539']

In [75]:
summary_docs

[Document(metadata={'doc_id': '0b537407-338a-400c-9299-269dfeda113d', 'text': 'We Build Trades, founded in 2017, is a UK-based digital marketing and software agency specializing in providing comprehensive solutions for trade businesses in the UK, Ireland, the US, and Canada. They offer services like website design, copywriting, automated systems (including Trades Gang), data & tracking analytics, paid advertising, and SEO to help businesses manage sales, marketing, and operations effectively. Their focus is on creating custom, conversion-focused strategies to help trade businesses stand out and improve their overall performance.'}, page_content='We Build Trades, founded in 2017, is a UK-based digital marketing and software agency specializing in providing comprehensive solutions for trade businesses in the UK, Ireland, the US, and Canada. They offer services like website design, copywriting, automated systems (including Trades Gang), data & tracking analytics, paid advertising, and SEO

In [ ]:
# # We can also add the original chunks to the vectorstore if we so want
# for i, doc in enumerate(docs):
#     doc.metadata[id_key] = doc_ids[i]
# retriever.vectorstore.add_documents(docs)

In [60]:
sub_docs = retriever.vectorstore.similarity_search("Founder of We Build Trade")

sub_docs[0]

Document(id='516fb7e3-8392-48c1-a95c-d2bc01e5064f', metadata={'doc_id': '27e4d90a-f2aa-487c-bd6f-0f3ea264496b'}, page_content='This document describes We Build Trades, a marketing agency founded in 2017 by Daniel Brown, a former marketing consultant. Brown\'s "relentless ambition" has driven the agency\'s growth, serving over 60 clients and expanding internationally to the UK, Ireland, US, and Canada. We Build Trades differentiates itself through its exclusive focus on trade businesses, international reach, and proprietary tools like Trades Gang and advanced analytics. It targets trade business owners seeking centralized marketing and operations solutions, growth-focused entrepreneurs looking to expand and streamline, and those wanting to enhance their branding through professional design, SEO, and advertising.')

In [62]:
from langchain_core.runnables import RunnablePassthrough

# Prompt template
template = """Answer the question based only on the following context, which can include text and tables:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)


# RAG pipeline
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [63]:
retriever.invoke("Founder of We Build Trade")

[Document(metadata={'source': '/content/Bot Name Jasmine.txt'}, page_content='4. Relentless Ambition\no Continuously striving for excellence and improvement.\nFounder: Daniel Brown\n• Role: Founder and CEO of We Build Trades.\n• Career Start: Former senior marketing consultant.\n• Journey: Founded We Build Trades in 2017 with limited resources but strong mentorship and a \npassion for learning.\n• Achievements: Under his leadership, the agency has served over 60 clients and expanded \ninternationally.\nKey Differentiators\n1. Specialization: Exclusive focus on trade businesses.\n2. International Reach: Services available across the UK, Ireland, the US, and Canada.\n3. Proprietary Tools: Unique solutions like Trades Gang and advanced analytics.\nUse Cases\n• For Trade Business Owners: Looking for a centralized solution to manage marketing and \noperations.\n• For Growth-Focused Entrepreneurs: Seeking tools and strategies to expand and streamline their \nbusinesses.\n• For Branding Enhan

In [64]:
result = chain.invoke(
    "Founder of We Build Trade"
)

In [65]:
import markdown
from IPython.display import display, Markdown

markdown_text = markdown.markdown(result)
display(Markdown(markdown_text))

<p>Daniel Brown</p>

In [66]:
result = chain.invoke(
    "Give me success story of  Daniel Lambert"
)

markdown_text = markdown.markdown(result)
display(Markdown(markdown_text))

<p>Daniel Lambert achieved a business transformation, turning it into a "lead-generating marketing machine" and praised the agency for exceeding his expectations by creating a high-performing lead-generating website.</p>

In [68]:
a = retriever.invoke("Give me success story of  Daniel Lambert")

In [70]:
a

[Document(metadata={'source': '/content/Bot Name Jasmine.txt'}, page_content='o Focus on continuous self-improvement and maintaining high standards.\nClient Success Stories\nWe Build Trades has a proven track record of helping clients achieve remarkable growth and business \ntransformation. Below are some notable success stories from clients who have benefited from their \ntailored digital marketing solutions and support.\nClient Success Stories\n1. Daniel Lambert\no Achieved a business transformation, turning it into a "lead-generating marketing \nmachine."\no Praised the agency for exceeding his expectations by creating a high-performing lead\x02generating website.\n2. Ashley Hale\no Sold 10 boilers in just four days after partnering with We Build Trades.\no Highlighted the team\'s professionalism and support as key factors in achieving this rapid \ngrowth.\n3. Alex Slater\no Secured over 20 inquiries and five jobs in just 15 days, generating £13,800.\no Credited the agency\'s user-f

In [77]:
len(a)

4